In [17]:


!pip install pyautogen[retrievechat]==0.2.0b5
!pip install amazon-codewhisperer-jupyterlab-ext

  Using cached sentence_transformers-2.2.2-py3-none-any.whl
     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/45.9 kB ? eta -:--:--
     ---------------------------------------- 45.9/45.9 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/123.5 kB ? eta -:--:--
     ---------------------------------- --- 112.6/123.5 kB 3.3 MB/s eta 0:00:01
     -------------------------------------- 123.5/123.5 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
  

In [18]:
import logging
import os

from autogen import config_list_from_dotenv
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent
from autogen import UserProxyAgent, AssistantAgent, GroupChat, GroupChatManager


In [19]:
config_list_default = config_list_from_dotenv(
    dotenv_file_path=".env",
    filter_dict={
        "model":{"gpt-4",
                 "gpt-3.5-turbo"}
    }
)
config_list_cheap = config_list_from_dotenv(
    dotenv_file_path=".env",
    filter_dict={
        "model":{"gpt-3.5-turbo"}
    }
)
config_list_costly = config_list_from_dotenv(
    dotenv_file_path=".env",
    filter_dict={
        "model":{"gpt-4"}
    }
)


In [ ]:
assistant_gpt35 = autogen.AssistantAgent(
    name="assistant_one",
    llm_config={
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list_cheap,
        "temperature"：0
    }
)

# creates new assistant using Assistant API
gpt_assistant = GPTAssistantAgent(
    name="assistant",
    llm_config={
        "config_list": config_list_costly,
        "assistant_id": None
    })

Product manager and coder

In [23]:


llm_config = {"config_list": config_list_cheap,
                "seed": int(os.getenv("seed")),
                "timeout": int(os.getenv("request_timeout"))}


user_proxy = UserProxyAgent(name="user_proxy",
                            system_message="A human adimin.",
                            max_consecutive_auto_reply=10,
                            is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
                            code_execution_config={
                                "work_dir": "coding",
                                "use_docker":False,
                            },
                            human_input_mode="NEVER")

pm = AssistantAgent(name="Product_manager",
                    system_message="Creative in software product ideas. Don't write any code. Just lay out the plan for what the"
                                    "coder need to code.",
                    llm_config=llm_config)


coder = AssistantAgent(
    name="coder",
    system_message=f"I am a 10x engineer, trained in Python. I was the first engineer at Uber" 
                   f"(which I make sure to tell everyone I work with).\n"
                   f"You are participating in a group chat with a user ({user_proxy.name})"
                   f"and a product manager ({pm.name}).",
    llm_config=llm_config
                   
)

groupchat = GroupChat(agents=[user_proxy, pm, coder], 
                      messages=[], 
                      max_round=12)
manager = GroupChatManager(name="manager", 
                           groupchat=groupchat, 
                           llm_config=llm_config)

user_proxy.initiate_chat(manager, 
                         message="Can you create Python code that generates a random number?")

user_proxy (to manager):

Can you create Python code that generates a random number?

--------------------------------------------------------------------------------
coder (to manager):

Sure, here is a simple way to generate a random number in Python using the `random` library:

```python
import random

def generate_random_num():
  random_num = random.randint(0, 100) # generates a random number between 0 and 100
  return random_num

print(generate_random_num())
```

This code defines a function `generate_random_num()` that generates a random number between 0 and 100 (you can adjust this range to whatever you need) and then prints that number.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...


user_proxy (to manager):

exitcode: 0 (execution succeeded)
Code output: 
90


--------------------------------------------------------------------------------
user_proxy (to manager):



--------------------------------------------------------------------------------
Product_manager (to manager):

Plan:
1. Define a function `generate_primes` which will take an integer `n` as an input argument. The function will generate all prime numbers up to `n`.
    - In the function, create a list `primes` to hold the generated prime numbers.
    - Implement a nested loop where the outer loop goes through each number from `2` up to `n` (inclusive), and the inner loop goes through each number in `primes`.
    - In the inner loop, check if the current number from the outer loop is divisible by the current number from `primes`. If so, break the loop; it's not a prime number. If not, it is a prime number, so append it to `primes`.
2. Define a function `generate_random_prime` which will take an integer

user_proxy (to manager):

exitcode: 0 (execution succeeded)
Code output: 
31


--------------------------------------------------------------------------------
user_proxy (to manager):



--------------------------------------------------------------------------------
Product_manager (to manager):

Plan:
- Our software will be a web-based language learning application that helps people learn new languages by immersing them in real-world scenarios.
- The application will include three main components:
    
	1. Frontend: This will be the user interface of the application where the users will interact. It will include the following screens:
	
		- Registration/Login: Users will be able to create a new account or log into an existing one.
		- Dashboard: Once logged in, users will see an overview of their progress, upcoming lessons, etc.
		- Lesson: This is where the actual learning takes place. Each lesson will have a real-world scenario like ordering food at a restaurant, asking for direct

In [25]:
user_proxy.chat_messages

defaultdict(list,
            {<autogen.agentchat.groupchat.GroupChatManager at 0x17a1ef8f0d0>: [{'content': 'Can you create Python code that generates a random number?',
               'role': 'assistant'},
              {'content': 'Sure, here is a simple way to generate a random number in Python using the `random` library:\n\n```python\nimport random\n\ndef generate_random_num():\n  random_num = random.randint(0, 100) # generates a random number between 0 and 100\n  return random_num\n\nprint(generate_random_num())\n```\n\nThis code defines a function `generate_random_num()` that generates a random number between 0 and 100 (you can adjust this range to whatever you need) and then prints that number.',
               'name': 'coder',
               'role': 'user'},
              {'content': 'exitcode: 0 (execution succeeded)\nCode output: \n90\n',
               'role': 'assistant'},
              {'content': '', 'role': 'assistant'},
              {'content': "Plan:\n1. Define a fun

Analyse RBA 

In [12]:
llm_config = {"config_list": config_list_default,
                "seed": int(os.getenv("seed")),
                "timeout": int(os.getenv("request_timeout"))}


user_proxy = UserProxyAgent(name="user_proxy",
                            system_message='''A human admin. Interact with the planner to discuss the plan. 
                            Plan execution needs to be approved by this admin.''',
                            max_consecutive_auto_reply=10,
                            is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
                            code_execution_config=False,
                            #human_input_mode="NEVER",
                            )

planner = AssistantAgent(name="Planner",
                    system_message='''Planner. Suggest a plan. 
                                        Revise the plan based on feedback from admin and critic, until admin approval.
                                        The plan may involve an engineer who can write code, an executor who can execute the code to get result, and a economist who analyses the result.
                                        A critic is also a member of the team who can provide feedback to plan, code, result of execution, and analysis by economist. 
                                        Explain the plan first. 
                                        Be clear which step is performed by which memeber. All members can proivde comment and opinion. 
                                        
''',
                    llm_config=llm_config)


engineer = AssistantAgent(
    name="Engineer",
    system_message=f"You are a 10x engineer, trained in Python. You analyse requirement, and write Python script. If the requirement is not clear, you ask for clarification."
                    f"If the requirement involves scraping web pages, your code saves scrapped text to one or multiple local files locally, each file is not over 2000 token limit." 
                   f"You don't execute code. You give your script to an exectuor to execute.\n"
                   f"If the executor reports error or failure of execution, you debug and fix your script."
                   f"If you find the requirement is not achiveable, let planner and other members know.",

    llm_config=llm_config
                   
)

critic = AssistantAgent(
    name="Critic",
    system_message=f"Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL. "
                    f"You don't write or execute code, don't analyse document directly. You provide feedback to members specialized at these tasks.",
    llm_config=llm_config,
)

executor = UserProxyAgent(
    name="Executor",
    system_message=f"Executor. Execute the code written by the engineer, save the result in one or multiple files locally."
                    f"If execution succeeds, you read out the texts in saved files to other members. If multiple files are saved, you read them one by one."
                    f"If execution fails, or result is empty, make sure to raise the issue to planner or engineer.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "work_dir"},
)

economist = AssistantAgent(
    name="economist",
    system_message=f"Economist. Provide economic insights. "
                    f"You read and analyse one or multiple texts provided by executor, and provide your professional insights in reponse to user question."
                    f"Your analysis must be supported by the provided texts and your knowledge. You are forbidden to make up texts or fake analysis.",  
    llm_config=llm_config,
)

groupchat = GroupChat(agents=[user_proxy, planner, engineer, critic, executor, economist], 
                      messages=[], 
                      max_round=12)
manager = GroupChatManager(name="manager", 
                           groupchat=groupchat, 
                           llm_config=llm_config)

user_proxy.initiate_chat(manager, 
                         message="Can you analyse the material at link https://www.rba.gov.au/monetary-policy/rba-board-minutes/2023/2023-11-07.html, and tell me whether the Australian Reserve Bank will increase interest rate in next Month or not?")

user_proxy (to manager):

Can you analyse the material at link https://www.rba.gov.au/monetary-policy/rba-board-minutes/2023/2023-11-07.html, and tell me whether the Australian Reserve Bank will increase interest rate in next Month or not?

--------------------------------------------------------------------------------
Planner (to manager):

Plan:
1. Engineer: The engineer will start by writing a code to scrape the textual data from the provided URL. To accomplish this, they may use various programming languages like Python with libraries such as Beautiful Soup for web scraping and Requests for accessing the URL.

2. Executor: The executor will run the scraping code written by the engineer. The output of this step should be the raw textual data from the given URL.

3. Economist: This textual data will be reviewed by the economist, who will analyze the content to understand the monetary policy discussed in the document, specifically looking for any indications or trends that suggest th

Executor (to manager):

exitcode: 0 (execution succeeded)
Code output: 


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...


Executor (to manager):

exitcode: 0 (execution succeeded)
Code output: 


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...


Executor (to manager):

exitcode: 0 (execution succeeded)
Code output: 


--------------------------------------------------------------------------------
Executor (to manager):



--------------------------------------------------------------------------------
Critic (to manager):

Looking at the code provided by the Engineer, there is little for the Critic to specifically analyze as the code is straightforward with defined tasks. The Python script is used to extract the text from the web page using BeautifulSoup and the requests library. The text is then stored in a .txt file for review. 

However, I would propose a minor alteration. Currently, only the first 2000 tokens of the extracted text are saved to the 'rba_minutes.txt'. Given that we cannot definitively know the structure of the webpage and where the relevant information may be located, I would suggest that all scraped text be saved instead of limiting it to the first 2000 tokens. This will ensure no potentially crucial infor

Executor (to manager):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 12, in <module>
    text = scrape_website(url)
NameError: name 'scrape_website' is not defined


--------------------------------------------------------------------------------
Engineer (to manager):

I apologize for the confusion. The modified script failed to execute properly due to a NameError. The 'scrape_website' function was not included in the presented script. I will amend this and present the entire script including the function definition.

```python
import requests
from bs4 import BeautifulSoup

def scrape_website(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup.get_text()

def save_to_file(data, base_filename):
    chars_per_file = 2000
    # split the data into chunks of 2000 chars
    chunks = [data[i:i + chars_per_file] for i in range(0, len(data), chars_per_file)]
    
    for i, chunk in en

Executor (to manager):

exitcode: 0 (execution succeeded)
Code output: 


--------------------------------------------------------------------------------


In [22]:
import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb


assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=llm_config,
)



assistant.reset()
ragproxyagent.initiate_chat(assistant, problem="whether the Australian Reserve Bank will increase interest rate in next Month or not?")

Trying to create collection.


	 ...
INFO:autogen.retrieve_utils:Found 5 chunks.


doc_ids:  [['doc_3', 'doc_2', 'doc_1', 'doc_4', 'doc_0']]
Adding doc_id doc_3 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: whether the Australian Reserve Bank will increase interest rate in next Month or not?

Context is: 						had increased in preceding months to be 9&nbsp;per&nbsp;cent above the low point in February 2023.
						Nonetheless, housing loan commitments remained almost 30&nbsp;per&nbsp;cent below their peak in January
						2022, consistent with the effect of higher interest rates on maximum borrowing capacity.</p>
					<p>Households&#146; required mortgage payments had increased further over the September quarter to around
						10&nbsp;per&nbsp;cent of household dis

doc_ids:  [['doc_3', 'doc_2', 'doc_1', 'doc_4', 'doc_0']]
doc_ids:  [['doc_3', 'doc_2', 'doc_1', 'doc_4', 'doc_0']]


doc_ids:  [['doc_3', 'doc_2', 'doc_1', 'doc_4', 'doc_0']]


doc_ids:  [['doc_3', 'doc_2', 'doc_1', 'doc_4', 'doc_0']]
No more context, will terminate.
ragproxyagent (to assistant):

TERMINATE

--------------------------------------------------------------------------------


In [35]:
llm_config = {"config_list": config_list_default,
                "seed": int(os.getenv("seed")),
                "timeout": int(os.getenv("request_timeout"))}

termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

user_proxy = UserProxyAgent(name="user_proxy",
                            system_message='''A human admin. Interact with the planner to discuss the plan. 
                            Plan execution needs to be approved by this admin.''',
                            max_consecutive_auto_reply=10,
                            is_termination_msg=termination_msg,
                            code_execution_config=False,
                            human_input_mode="TERMINATE",
                            )

planner = AssistantAgent(name="Planner",
                    system_message='''Planner. Suggest a plan. 
                                        Revise the plan based on feedback from admin and critic, until admin approval.
                                        The plan may involve a document management specialist who retrieves relevant documents, a economist who analyses the result, 
                                        and a critic who provides feedback to planner's plan, documents retrieved by document management specialist, and analysis produced by economist. 
                                        Explain the plan first. 
                                        Be clear which step is performed by which memeber. All members can proivde comment and opinion. 
                                        ''',
                    llm_config=llm_config)

document_maagement_specialist = RetrieveUserProxyAgent(
    name="document_management_specialist",
    system_message="Document Management Specialist. Retrieve relevant documents from the internet.",
    is_termination_msg=termination_msg,
    retrieve_config={
        "task": "qa",
        "docs_path": "data/rba_minutes.txt", #"https://www.rba.gov.au/monetary-policy/rba-board-minutes/2023/2023-11-07.html",
        "chunk_token_size": 1000,
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "collection_name": "groupchat",
        "get_or_create": True,
    },
    code_execution_config=False,
    human_input_mode="NEVER",
)


critic = AssistantAgent(
    name="Critic",
    system_message=f"Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL. "
                    f"You don't write or execute code, don't analyse document directly. You provide feedback to members specialized at these tasks.",
    llm_config=llm_config,
)


economist = RetrieveAssistantAgent(
    name="economist",
    system_message=f"Economist. Provide economic insights. "
                    f"You read and analyse one or multiple texts provided by executor, and provide your professional insights in reponse to user question."
                    f"Your analysis must be supported by the provided texts and your knowledge. You are forbidden to make up texts or fake analysis.",  
    llm_config=llm_config,
)



def retrieve_content(message, n_results=3):
        document_maagement_specialist.n_results = n_results  # Set the number of results to be retrieved.
        # Check if we need to update the context.
        update_context_case1, update_context_case2 = document_maagement_specialist._check_update_context(message)
        if (update_context_case1 or update_context_case2) and document_maagement_specialist.update_context:
            document_maagement_specialist.problem = message if not hasattr(document_maagement_specialist, "problem") else document_maagement_specialist.problem
            _, ret_msg = document_maagement_specialist._generate_retrieve_user_reply(message)
        else:
            ret_msg = document_maagement_specialist.generate_init_message(message, n_results=n_results)
        return ret_msg if ret_msg else message


for agent in [user_proxy, planner, economist, critic]:
    # register functions for all agents.
    agent.register_function(
        function_map={
            "retrieve_content": retrieve_content,
        }
    )

groupchat = GroupChat(agents=[user_proxy, planner, document_maagement_specialist, critic, economist], 
                      messages=[], 
                      max_round=12)
manager = GroupChatManager(name="manager", 
                           groupchat=groupchat, 
                           llm_config=llm_config)

# user_proxy.initiate_chat(manager, 
#                          message="Whether Australian Reserve Bank will increase interest rate in next Month or not?")
document_maagement_specialist.initiate_chat(manager, 
                         problem="Whether Australian Reserve Bank will increase interest rate in next Month or not?")

SyntaxError: invalid syntax (1975692970.py, line 32)

In [9]:
gpt_assistant.delete_assistant()

     ---------------------------------------- 0.0/115.7 kB ? eta -:--:--
     ------------------------------------ - 112.6/115.7 kB 3.3 MB/s eta 0:00:01
     -------------------------------------- 115.7/115.7 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached jupyterlab-4.0.9-py3-none-any.whl.metadata (15 kB)
  Using cached async_lru-2.0.4-py3-none-any.whl.metadata (4.5 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached jupyter_lsp-2.2.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached jupyter_server-2.10.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached jupyterlab_server-2.25.2-py3-none-any.whl.meta

In [ ]:
from openai import OpenAI
client = OpenAI()

audio_file= open("/path/to/file/audio.mp3", "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)

In [ ]:
#function that adds two numbers

def add(a,b):
    return a+b

#write a test casec for the function


In [3]:
!pip install pandas

     ---------------------------------------- 0.0/341.8 kB ? eta -:--:--
     ------- ------------------------------- 61.4/341.8 kB 3.2 MB/s eta 0:00:01
     ------------------ ------------------- 163.8/341.8 kB 2.5 MB/s eta 0:00:01
     ------------------ ------------------- 163.8/341.8 kB 2.5 MB/s eta 0:00:01
     ------------------ ------------------- 163.8/341.8 kB 2.5 MB/s eta 0:00:01
     -------------------------------------  337.9/341.8 kB 1.6 MB/s eta 0:00:01
     -------------------------------------  337.9/341.8 kB 1.6 MB/s eta 0:00:01
     -------------------------------------- 341.8/341.8 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.7 MB 3.6 MB/s eta 0:00:03
    --------------------------------------- 0.2/10.7 MB 2.8 MB/s eta 0:00:04
   - -------------------------------------- 0.3/10.7 MB 2.4 MB/s eta 0:00:05
   - -------------------------------------- 0.4/10.7 MB 2.4 MB/s e

In [7]:
!pip install seaborn

     ---------------------------------------- 0.0/157.7 kB ? eta -:--:--
     -------------------- ------------------ 81.9/157.7 kB 2.3 MB/s eta 0:00:01
     -------------------------------------- 157.7/157.7 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
   ---------------- ----------------------- 122.9/294.6 kB 3.6 MB/s eta 0:00:01
   ----------------------- ---------------- 174.1/294.6 kB 2.6 MB/s eta 0:00:01
   -------------------------------------- - 286.7/294.6 kB 2.5 MB/s eta 0:00:01
   -------------------------------------- - 286.7/294.6 kB 2.5 MB/s eta 0:00:01
   ---------------------------------------- 294.6/294.6 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.6 MB 2.0 MB/s eta 0:00:04
   ---------------------------------------- 0.1/7.6 MB 2.3 MB/s eta 0:00:04
   - -------------------------------------- 0.3/7.6 MB 2.3 MB/s eta 0:00:04

In [11]:
!pip install scipy

     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 60.4/60.4 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/44.1 MB 3.3 MB/s eta 0:00:14
   ---------------------------------------- 0.2/44.1 MB 2.9 MB/s eta 0:00:16
   ---------------------------------------- 0.4/44.1 MB 2.9 MB/s eta 0:00:16
   ---------------------------------------- 0.5/44.1 MB 3.0 MB/s eta 0:00:15
   ---------------------------------------- 0.5/44.1 MB 3.0 MB/s eta 0:00:15
    --------------------------------------- 0.7/44.1 MB 2.6 MB/s eta 0:00:18
    --------------------------------------- 0.8/44.1 MB 2.6 MB/s eta 0:00:17
    --------------------------------------- 0.9/44.1 MB 2.6 MB/s eta 0:00:17
    --------------------------------------- 1.1/44.1 MB 2.7 MB/s eta 0:00:17
   - -------------------------------------- 1.1/44.1 MB 2.6 MB/s eta 0:00:17
   - ---

In [5]:
import pandas as pd
titanic_data = pd.read_csv('D:/Documents/data/titanic/train.csv')


In [9]:
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
import seaborn as sns
correlation_matrix = titanic_data.corr()

sns.heatmap(correlation_matrix, annot=True)

ValueError: could not convert string to float: 'Braund, Mr. Owen Harris'

In [12]:
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import numpy as np

# Load the Titanic dataset (replace with the path to your actual file)
titanic_data = sns.load_dataset('titanic')

# Identify numerical and categorical columns
numerical_cols = titanic_data.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = titanic_data.select_dtypes(include=['object', 'category']).columns

# Compute correlations between numerical columns
correlation_matrix = titanic_data[numerical_cols].corr()

# Function to compute Cramer's V for categorical variables
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2_corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    r_corr = r - ((r-1)**2)/(n-1)
    k_corr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2_corr / min((k_corr-1), (r_corr-1)))

# Analyze associations between categorical columns
cramers_results = {}
for col1 in categorical_cols:
    for col2 in categorical_cols:
        if col1 != col2:
            cramers_results[col1 + " vs " + col2] = cramers_v(titanic_data[col1], titanic_data[col2])

# Print results
print("Correlation matrix for numerical variables:\n", correlation_matrix)
print("\nCramer's V results for categorical variables:")
for pair, v in cramers_results.items():
    print(pair, ":", v)


Correlation matrix for numerical variables:
           survived    pclass       age     sibsp     parch      fare
survived  1.000000 -0.338481 -0.077221 -0.035322  0.081629  0.257307
pclass   -0.338481  1.000000 -0.369226  0.083081  0.018443 -0.549500
age      -0.077221 -0.369226  1.000000 -0.308247 -0.189119  0.096067
sibsp    -0.035322  0.083081 -0.308247  1.000000  0.414838  0.159651
parch     0.081629  0.018443 -0.189119  0.414838  1.000000  0.216225
fare      0.257307 -0.549500  0.096067  0.159651  0.216225  1.000000

Cramer's V results for categorical variables:
sex vs embarked : 0.11307233125458868
sex vs class : 0.12968965030528873
sex vs who : 0.9470211954532946
sex vs deck : 0.23786240521276916
sex vs embark_town : 0.11307233125458868
sex vs alive : 0.5401999468101071
embarked vs sex : 0.11307233125458868
embarked vs class : 0.25981056003546527
embarked vs who : 0.0792568531584825
embarked vs deck : 0.1468252267455849
embarked vs embark_town : 1.0
embarked vs alive : 0.166058